In [22]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [57]:
def load_data(folder_path, num_images=None):
    images = []

    for filename in os.listdir(folder_path):
        if num_images is not None and len(images) >= num_images:
            break

        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  
        images.append(img)

    return np.array(images)

In [58]:
def build_and_train_autoencoder(X_train, X_test):
    model = models.Sequential([
        layers.Input(shape=(64, 64, 3)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Output with 3 channels for color images
    ])

    model.compile(optimizer='adam', loss='mse')  # Using mean squared error for color images

    model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))
    
    return model

In [59]:
folder_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"
X = load_data(folder_path, num_images=10000)
print(len(X))

10000


In [60]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 64, 64, 3)  
X_test = X_test.reshape(-1, 64, 64, 3)

In [61]:
autoencoder_model = build_and_train_autoencoder(X_train, X_test)
autoencoder_model.save("autoencoder_model.h5")

Epoch 1/10
250/250 [==============================] - 40s 157ms/step - loss: 0.0325 - val_loss: 0.0167
Epoch 2/10
250/250 [==============================] - 36s 144ms/step - loss: 0.0148 - val_loss: 0.0131
Epoch 3/10
250/250 [==============================] - 36s 144ms/step - loss: 0.0127 - val_loss: 0.0117
Epoch 4/10
250/250 [==============================] - 36s 144ms/step - loss: 0.0114 - val_loss: 0.0113
Epoch 5/10
250/250 [==============================] - 34s 137ms/step - loss: 0.0106 - val_loss: 0.0116
Epoch 6/10
250/250 [==============================] - 36s 142ms/step - loss: 0.0100 - val_loss: 0.0108
Epoch 7/10
250/250 [==============================] - 34s 137ms/step - loss: 0.0095 - val_loss: 0.0092
Epoch 8/10
250/250 [==============================] - 35s 142ms/step - loss: 0.0090 - val_loss: 0.0089
Epoch 9/10
250/250 [==============================] - 35s 138ms/step - loss: 0.0086 - val_loss: 0.0085
Epoch 10/10
250/250 [==============================] - 36s 144ms/step - l

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [75]:
def classify_large_image(autoencoder_model, larger_image_path, threshold=0.1):
    larger_image = cv2.imread(larger_image_path)
    larger_image_resized = cv2.resize(larger_image, (64, 64)) / 255.0
    larger_image_input = np.expand_dims(larger_image_resized, axis=0)

    prediction = autoencoder_model.predict(larger_image_input)

    # Assuming autoencoder_model output has shape (batch_size, 64, 64, 3)
    reconstruction_error = np.mean(np.abs(larger_image_input - prediction))

    if reconstruction_error > threshold:
        print("Face detected!")
    else:
        print("No face detected.")

autoencoder_model = load_model("autoencoder_model.h5")
larger_image_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_photos/no_face.jpg"
classify_large_image(autoencoder_model, larger_image_path, threshold=0.085)


1/1 [==============================] - 0s 50ms/step
No face detected.
